- Give the objective of k-means
- Give the k-means algorithm
- run k-means on several examples visually. Show examples of global minimum and local minimum. resuly is sensitive to initial centroids.
- K-means++, Bisecting K-means
- Running time of k-means
- Limitations of k-means: spherical clusters assumption, show examples where k-means have a problem: different sized clusters, hard assignments

### Cluster Evaluation
- For the same number of clusters, the objective function can be used. But don't forget that there can be no objective measure since clustering is in many cases subjective.
- But as the number of clusters increase, the objective goes to zero, comes the elbow technique.


## Mixture of Gaussians

- Take 3 uni-variate normal distributions. Generate some data. 
- Question: Given the data can we discover that there are 3 distributions and their mean and stds.